# Lab: Naive Bayes


Naive Bayes is a time tested machine learning algorithm that is mostly used for text classification. A famous usage of naive Bayes is for spam filtering.

![ETL](https://www.unlocktheinbox.com/images/resource_spam_filters.png)


The goal of this lab session is to use two variations of the **naive Bayes** algorithm. Each variation has its specific strengths.

The **first** variation is the **Bernoulli naive Bayes** version and is given below. You will have to go through the explanation and code to understand how it works.

In the **second** notebook you will have to code **multinomial naive Bayes** yourself.

In the **third** notebook you will have to apply the **multinomial naive Bayes** algorithm on a **real-world dataset**. The goal is to automatically determine if a movie review is positive or negative. As this is not a toy dataset, you'll have to **optimize the code** or it will have to run for a long time

In the **fourth** notebook you will have to extend the **multinomial naive Bayes** algorithm to work with **more than two classes**.

In the **fifth** notebook you will have to apply the **bernouilli naive Bayes** algorithm on **real-world movie reviews dataset** set. Hence, the **code** will have to be **optimized** too.

All notebooks should be uploaded before the start of the next lab session.

### These exercises should be done individually. This means a submission per student is required. Plagiarism will result in a 0 for the plagiarised parts.

## Part1: Bernoulli naive Bayes

The Bernoulli naive Bayes algorithm works as follows:

We want to know what the probability is of a class given a document: $P(C|D)$
wherein $D$ is the document and $C$ a class.

To calculate this probability Bayes' rule is applied.

\begin{equation*}
P(C|D) = \frac{P(D|C)P(C)}{P(D)} \propto P(D|C)P(C)
\quad\quad\text{(1)}
\end{equation*}


- $P(C)$ is called the *prior* and expresses our initial believes of a class occuring in the dataset.
- $P(D|C)$ is called the *likelihood* and expresses the probability of observing that document given the class.
- The output $P(C|D)$ is called the *posterior*. Hence by applying Bayes' rule we update our initial believes by observing the data.

In (1), $\propto$ means "proportial to". We do not calculate P(D) as it is just a scaling factor.

Throughout this notebook these different probabilities will be calculated.


**In this notebook 4 steps are done:**

1. Data set creation
2. Prior calculation
3. Likelihood calculation
4. Classification (posterior calculation)

#### Import(s)

In [1]:
from __future__ import print_function
import numpy as np

### 1. Data set creation

We start of with a small data set:

- *class_vec* contains the labels of the samples
- *sentences* is a list that contains lists of strings (sentences)

In [2]:
class_vec = np.array([0,1,0,1,0,1])
sentences = np.array([['my', 'dog', 'has', 'flea', 'problems', 'help', 'please','help'],
             ['maybe', 'stop', 'taking', 'him', 'to', 'dog', 'park', 'stupid'],
             ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
             ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
             ['mr', 'licks', 'ate', 'my', 'steak', 'how','to', 'stop', 'him'],
             ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']])

#### 1.1 From text to feature vectors

To transform text into feature vector that can be used by the algorithm, the bag of words (BOW) approach is used.

**BOW works as follows**:

- Get all unique words in the dataset to create a vocabulary ($V$).
- For every sentence in the dataset create a binary (Bernoulli) list/array/vector with length equal to the number of elements in the vocabulary ($|V|$).
- For every word in a given sentence a *1* is set in the feature vector if it is present in the vocabulary else a *0* is set.

**Example**:

$V$ = [blue, red, dog, cat, biscuit, apple]

The length of $V$ (denoted by $|V|$) is equal to 6.

Hence the sentence "the blue dog ate a blue biscuit" is encoded as ${\bf{x}}$=[1,0,1,0,1,0].

Note that even though the word *blue* occurs two times, the binary vector solely indicates its presence.

#### 1.2 Vocabulary creation

In [3]:
# List which will contain all unique words in the data set
all_words = []

# Transform the data set (which is a list of lists) to a single list
for sentence in sentences:
    all_words.extend(sentence)

# Use the numpy function "unique" to get all unique elements from a list
vocab = np.unique(all_words)

print(vocab)

['I' 'ate' 'buying' 'cute' 'dalmation' 'dog' 'flea' 'food' 'garbage' 'has'
 'help' 'him' 'how' 'is' 'licks' 'love' 'maybe' 'mr' 'my' 'park' 'please'
 'posting' 'problems' 'quit' 'so' 'steak' 'stop' 'stupid' 'taking' 'to'
 'worthless']


#### 1.3 Sentences encoding

In [4]:
def encode_binary(vocab,sentence):
    vocab_list = vocab.tolist()
    
    # Create the binary feature vector and initialize every element as 0
    binary_sentence=np.zeros(len(vocab_list),)
    
    # For every word in the sentence check if it is in the vocabulary,
    # if it is: set a 1 on the positions of that element in the binary feature vector
    for word in sentence:
        if word in vocab:
            binary_sentence[vocab_list.index(word)]=1.
    return binary_sentence

#### 1.4 Data set transformation

In [5]:
# apply the function defined above to every sentence in the data set
data_set = []
for sentence in sentences:
    binary_sentence = encode_binary(vocab, sentence)
    data_set.append(binary_sentence)
    
data_set = np.array(data_set)
print(data_set)

[[0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 1. 1. 1. 0.]
 [1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 1. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 1. 0. 0. 1.]]


In [6]:
print("The first sentence:")
print(sentences[0])
print()
print("A check to see if the sentence was properly encoded")
print(vocab[data_set[0]==1.0])

The first sentence:
['my', 'dog', 'has', 'flea', 'problems', 'help', 'please', 'help']

A check to see if the sentence was properly encoded
['dog' 'flea' 'has' 'help' 'my' 'please' 'problems']


### 2. Prior calculation

As prior we calculate the fraction of documents belonging to each class.
As there are 6 documents in total and 3 belonging to class 0 (normal) and 3 beloging to class 1 (rude):

- $P(C=0)$ = $3/6$
- $P(C=1)$ = $3/6$

In [7]:
# Total number of sentences
N = np.float(len(sentences))

prior_0 = len(class_vec[class_vec==0])/N
prior_1 = len(class_vec[class_vec==1])/N

print("Prior for 0: ", prior_0)
print("Prior for 1: ", prior_1)

Prior for 0:  0.5
Prior for 1:  0.5


### 3. Likelihood calculation

$P(D_i|C)$ is the likelihood of the ith document ($D_i$) in the dataset, given the class $C$.

The i-th document is encoded as the feature vector $\textbf{x}_{i}$.

For Bernouilli naive Bayes the likelihood is:

\begin{equation*}
P(D_i|C)\approx P(\textbf{x}_{i}|C) = \prod_{t=1}^{|V|}\Bigl(P(w_t|C)^{x_{it}}(1-P(w_t|C))^{(1-x_{it})}\Bigr)
\quad\quad\text{(2)}
\end{equation*}

- $w_t$ is the t-th element (word) in a document
- $P(w_t|C)$ is the probability of word $w_t$ occurring in a document of class C
- $x_{it}$ is the t-th element in the feature vector $\textbf{x}_i$

This product goes over all words in the vocabulary. If word $w_t$ is present in $D_i$, then $x_{it} = 1$ and the probability of that word occuring in class C is $P(w_t |C)$; if word $w_t$ is not present in $D_i$, then $x_{it} = 0$ and the propability of that word not occuring in C is $1 − P(w_t |C)$. **As can be seen, the non-occurance of a word is also taken into account.**

To estimate $P(w_t |C)$ we calculate in how many documents, of the given class ($k$), the word $w_t$ is observed (denoted by $n_k(w_t))$. Afterwards, normalization is applied by dividing by the total number of documents belonging to that class (denoted as $N_k$):

\begin{equation*}
P(w_t |C=k) = \frac{1+n_k(w_t)}{2+N_k}
\quad\quad\text{(3)}
\end{equation*}

Laplace smoothing is also used in (3) so that the likelihood can't be zero.

First $P(w_t |C=k)$  is calculated:

In [8]:
# For each word, we want to know in how many documents of a certain class it occured
# +1 for the smoothing
word_count_class_0 = np.sum(data_set[class_vec==0],axis=0) + 1
word_count_class_1 = np.sum(data_set[class_vec==1],axis=0) + 1

# For each class we want to know how many documents belong to it 
# +2 for laplacian smoothing
doc_count_class_0 = len(class_vec[class_vec==0]) + 2
doc_count_class_1 = len(class_vec[class_vec==1]) + 2

# Multiply by 1. to force conversion to floating number
words_likelihood_0 = 1. * word_count_class_0 / doc_count_class_0
words_likelihood_1 = 1. * word_count_class_1 / doc_count_class_1

print("words_likelihood_0:", words_likelihood_0)
print("words_likelihood_1:", words_likelihood_1)

words_likelihood_0: [0.4 0.4 0.2 0.4 0.4 0.4 0.4 0.2 0.2 0.4 0.4 0.6 0.4 0.4 0.4 0.4 0.2 0.4
 0.8 0.2 0.4 0.2 0.4 0.2 0.4 0.4 0.4 0.2 0.2 0.4 0.2]
words_likelihood_1: [0.2 0.2 0.4 0.2 0.2 0.6 0.2 0.4 0.4 0.2 0.2 0.4 0.2 0.2 0.2 0.2 0.4 0.2
 0.2 0.4 0.2 0.4 0.2 0.4 0.2 0.2 0.6 0.8 0.4 0.4 0.6]


### 4. Classification (posterior calculation)

Finally, to classify a (new) sentence we need the priors (P(C=0) and P(C=1)) and the likelihoods (P(D|C=0) and P(D|C=1)). The likelihood will be calculated next as described in equation 2.

As a probability can be a small number, and multiplying small numbers can lead to precision problems, we can use following rule:

\begin{equation*}
\log(uv) = \log(u) + \log(v)
\end{equation*}

We apply this rule on equation *(2)*, resulting in:


\begin{align}
P(D_i|C)\approx P(\textbf{x}_{i}|C) & = \prod_{t=1}^{|V|}\Bigl(P(w_t|C)^{x_{it}}(1-P(w_t|C))^{(1-x_{it})}\Bigr) \\
\log(P(D_i|C))\approx \log(P(\textbf{x}_{i}|C)) & = \sum_{t=1}^{|V|}\Bigl(x_{it}\log(P(w_t|C))+(1-x_{it})\log(1-P(w_t|C))\Bigr) \quad\quad\text{(4)}
\end{align}

 and subsequently on Bayes' rule itself to calculate the posterior:

\begin{equation*}
\log(P(C|D)) \propto \log(P(D|C))+\log(P(C))
\quad\quad\text{(5)}
\end{equation*}

Using the posterior, the sentence can then be classified as follows:

\begin{equation*}
\begin{cases}
      0, & \text{if}\ \log(P(C=0|D_i))> \log(P(C=1|D_i)) \\
      1, & \text{otherwise}
\end{cases}\quad\text{(6)}
\end{equation*}

In [9]:
def classify(sentence,vocab,words_likelihood_0,words_likelihood_1,prior_0,prior_1):

    # Create a BOW representation of the new sentence
    coded_sentence = encode_binary(vocab,sentence)

    # Apply equation (4) to get the likelihoods
    log_likelihood_0 = np.sum((coded_sentence*np.log(words_likelihood_0))+((1-coded_sentence)*np.log(1-words_likelihood_0))) # equation 4 where C=0 
    log_likelihood_1 = np.sum((coded_sentence*np.log(words_likelihood_1))+((1-coded_sentence)*np.log(1-words_likelihood_1))) # equation 4 where C=1 
    
    # Apply equation (5) to get the eventual results.
    posterior_0 = np.log(prior_0) + log_likelihood_0
    posterior_1 = np.log(prior_1) + log_likelihood_1
    
    # Classify according to equation (6)
    if posterior_0 > posterior_1:
        return 0
    else:
        return 1

### The two sentences:

In [10]:
classify(['my','dog','is','cute','he','licks','me'],vocab,words_likelihood_0,words_likelihood_1,prior_0,prior_1)

0

In [11]:
classify(['my','dog','is','stupid','and','worthless',"real"],vocab,words_likelihood_0,words_likelihood_1,prior_0,prior_1)

1